### Parse and convert protocol buffers

Inspired by
https://stackoverflow.com/questions/38958751/parsing-nyc-transit-mta-historical-gtfs-data-not-realtime
Data Source

This extracts data from the protobufs manually downloaded from [MTA Alert Archive](http://web.mta.info/developers/data/archives.html)the latest source suggested at:
https://groups.google.com/d/msg/mtadeveloperresources/Whm5XTVINcE/z-LO12ANAAAJ

Additional feeds are listed here:
http://web.mta.info/developers/developer-data-terms.html

Note that the above historical datasource is outdated, and the above MTA Alert Archive is correct

NOTE: This assumes that the protobufs have already been downloaded to <code>MTADelayPredict/data/raw/status</code> e.g. <code>MTADelayPredict/data/raw/status/201901.zip</code>

In [2]:
import os
data_dir = '../data/raw/status'

In [3]:
proto_file = os.path.join(os.path.join(data_dir), 'gtfs-realtime.proto')
! wget -O $proto_file https://developers.google.com/transit/gtfs-realtime/gtfs-realtime.proto

--2020-04-28 19:41:13--  https://developers.google.com/transit/gtfs-realtime/gtfs-realtime.proto
Resolving developers.google.com (developers.google.com)... 172.217.10.142, 2607:f8b0:4006:812::200e
Connecting to developers.google.com (developers.google.com)|172.217.10.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27065 (26K) [None]
Saving to: ‘../data/raw/status/gtfs-realtime.proto’

../data/raw/status/ 100%[===================>]  26.43K  --.-KB/s    in 0.02s   

2020-04-28 19:41:13 (1.51 MB/s) - ‘../data/raw/status/gtfs-realtime.proto’ saved [27065/27065]



In [4]:
proto_file = os.path.join(os.path.join(data_dir), 'gtfs-realtime.proto')
! wget -O $proto_file https://developers.google.com/transit/gtfs-realtime/gtfs-realtime.proto

--2020-04-28 19:41:14--  https://developers.google.com/transit/gtfs-realtime/gtfs-realtime.proto
Resolving developers.google.com (developers.google.com)... 172.217.10.142, 2607:f8b0:4006:812::200e
Connecting to developers.google.com (developers.google.com)|172.217.10.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27065 (26K) [None]
Saving to: ‘../data/raw/status/gtfs-realtime.proto’

../data/raw/status/ 100%[===================>]  26.43K  --.-KB/s    in 0.02s   

2020-04-28 19:41:14 (1.62 MB/s) - ‘../data/raw/status/gtfs-realtime.proto’ saved [27065/27065]



In [5]:
! protoc -I $data_dir --python_out=$data_dir $data_dir/nyct-subway.proto $data_dir/gtfs-realtime.proto

[libprotobuf WARNING google/protobuf/compiler/parser.cc:637] No syntax specified for the proto file: nyct-subway.proto. Please use 'syntax = "proto2";' or 'syntax = "proto3";' to specify a syntax version. (Defaulted to proto2 syntax.)


## Examine single delay case

Selected from [MTA Alert Archive](http://web.mta.info/developers/data/archives.html).  I also received an email alert for this, and it could have potentially impacted my commute if it had happened slightly earlier in the day.  We are going to begin this exercise focusing *exclusively* on northbound N trains.

![](files/20181221_0919_NR_Delay.png "20181221 Alert")

First, fetch the alert from the downloaded alert data.

### Load alert historical data

In [61]:
import pandas as pd

In [62]:
alert_dir = '../data/raw/alerts'
alert_df = pd.read_csv(os.path.abspath(os.path.join(alert_dir, 'raw_alerts_12.01.2018_12.31.2018.csv')))
alert_df.index = alert_df.Date.map(pd.to_datetime)
alert_df.drop(columns=['Date'], inplace=True)
alert_df.sort_index(inplace=True)

In [63]:
alert_df.loc[alert_df.Subject.str.match(r'.*N and R.*')]['2018-12-21 09:10':'2018-12-21 22:20']

,Agency,Subject,Message
Date,,,
2018-12-21 09:19:00,NYC,"BKLYN, N and R Trains, Delays",Northbound N and R trains are running with som...


#### No delay resolution message

This is an interesting example because we receive a delay message, however there is no subsequent resolution sent, as there are in some cases of actual stoppage e.g. "UPDATED: N and R Trains have resumed running with residual delays"

In [65]:
alert_df.loc[alert_df.Subject.str.match(r'UPDATE.*N and R.*')]['2018-12-21 09:10':'2018-12-21 22:20']

,Agency,Subject,Message
Date,,,


### Load subway gtfs statuses from this time period

In [8]:
import sys
sys.path.append(os.path.join(data_dir))
import nyct_subway_pb2
import gtfs_realtime_pb2

In [73]:
msg = gtfs_realtime_pb2.FeedMessage()

with open(os.path.join(data_dir, '201812/20181221/gtfs_nqrw_20181221_091908.gtfs'),'rb') as fh:
    msg.ParseFromString(fh.read())

In [ ]:
filter_dict = { trip:{ route_id } }

In [ ]:
def stop_times(msg, direction, route_id, stop_id):
    for e in msg.entity:
        trip_direction = e.trip_update.trip.Extensions[nyct_subway_pb2.nyct_trip_descriptor].direction
        trip_route_id = e.trip_update.trip.route_id

        # Only look for stop updates if this is for the right route and train
        if trip_direction == direction and trip_route_id == route_id:
            print trip_direction
            

In [143]:
time_square = ['R16', '127', '725']
direction = nyct_subway_pb2.NyctTripDescriptor.Direction.NORTH

e = msg.entity[0]
#for e in msg.entity:
#    if e.
#feed.entity[0].trip_update.trip.Extensions[nyct.nyct_trip_descriptor].direction
direction == e.trip_update.trip.Extensions[nyct_subway_pb2.nyct_trip_descriptor].direction

True

In [154]:
e.trip_update.trip.route_id

'N'

In [149]:
e.trip_update.stop_time_update[0]

arrival {
  time: 1545401933
}
departure {
  time: 1545401933
}
stop_id: "R04N"
schedule_relationship: SCHEDULED
[nyct_stop_time_update] {
  scheduled_track: "G2"
  actual_track: "G2"
}

In [127]:
msg.entity[0].trip_update.trip.Extensions[nyct_subway_pb2.nyct_trip_descriptor]

train_id: "1N 0758+ STL/DIT"
is_assigned: true
direction: NORTH

In [ ]:
nyct_subway_pb2.nyct_trip_descriptor.

In [8]:
import glob
protobuf_paths = glob.glob('{}/[0-9]*.zip'.format(data_dir))

if len(protobuf_paths) == 0:
    raise ValueError('No matching protbufs found in {}, please download from https://m.mymtaalerts.com/archive')
    
print(protobuf_paths)

['../data/raw/status/201811.zip', '../data/raw/status/201812.zip', '../data/raw/status/201901.zip', '../data/raw/status/201902.zip']


In [7]:
def gtfs_daterange():
    return 0

In [82]:
import zipfile
import shutil
import progressbar
import io

msg = gtfs_realtime_pb2.FeedMessage()

# Keep a list of files with failed conversions
failed_files = os.path.join(data_dir, 'failures.txt')

# unzip monthly rollups, then unzip the daily files inside
# This code is largely copied from: https://stackoverflow.com/questions/36285502/how-to-extract-zip-file-recursively-in-python
# The daily zipfiles are ~1GB, so there are big speed gains from unzipping in memory
for monthly_file in protobuf_paths:
    widgets = [progressbar.Percentage(), progressbar.Bar(), progressbar.Variable('failures')]    

    
    print("Extracting: " + monthly_file)
    z = zipfile.ZipFile(monthly_file)
    for i,f in enumerate(z.namelist()):
        print("{}/{}".format(i+1, len(z.namelist())))
        # get directory name from file
        dirname = os.path.join(data_dir, os.path.splitext(f)[0])
        # create new directory
        os.makedirs(dirname, exist_ok=True)
        # read inner zip file into bytes buffer 
        content = io.BytesIO(z.read(f))
        zip_file = zipfile.ZipFile(content)
        
        # Skip if already unzipped
        if not force:
            if len(glob.glob(dirname+'/*')) == len(zip_file.namelist()):
                print("Skipping " + os.path.basename(dirname))
                continue
         
        # Iterate through in-memory zipfile, decoding protobuf into json
        bar = progressbar.ProgressBar(widgets=widgets, max_value=len(zip_file.namelist()), min_poll_interval=.5).start()
        failures = 0
        for j,f2 in enumerate(zip_file.namelist()):
            
            try:
                # Dump the message into a json file for now
                msg.ParseFromString(zip_file.read(f2))

                # add message handler
                
            except Exception as e:
                # At the moment, some messages a sporadically unable to parse
                with io.open(failed_files, 'a') as fh:
                    fh.write(f2+'\n')
                    
                failures += 1
            
            # For now, just bail in order to examine the msg object
            raise Exception("Debug Exception")
            
            sys.stdout.flush()
            bar.update(j+1, failures=failures)
        zip_file.close()
        
        bar.finish()
    
    

Extracting: ../data/raw/status/201811.zip
1/30


N/A%|                                                         |failures: ------

Exception: Debug Exception

In [79]:
# What trains are available here
train_set = set()
for e in entity:
    train_set.append

header {
  gtfs_realtime_version: "1.0"
  incrementality: FULL_DATASET
  timestamp: 1541060371
  [nyct_feed_header] {
    nyct_subway_version: "1.0"
    trip_replacement_period {
      route_id: "J"
      replacement_period {
        end: 1541062171
      }
    }
    trip_replacement_period {
      route_id: "Z"
      replacement_period {
        end: 1541062171
      }
    }
  }
}
entity {
  id: "46000001"
  trip_update {
    trip {
      trip_id: "021500_J..N"
      start_date: "20181101"
      route_id: "J"
      [nyct_trip_descriptor] {
        train_id: "1J 0335 BRD/P-A"
        is_assigned: true
        direction: NORTH
      }
    }
    stop_time_update {
      arrival {
        time: 1541060354
      }
      departure {
        time: 1541060354
      }
      stop_id: "J17N"
      schedule_relationship: SCHEDULED
      [nyct_stop_time_update] {
        scheduled_track: "J1"
        actual_track: "J1"
      }
    }
    stop_time_update {
      arrival {
        time: 1541060444
 